In [2]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, sqrt, atan2

In [3]:
# This function defines the distance between two locations.
def distance(loc1, loc2, R = 6380):
    #change the value into radians
    lat1 = radians(float(loc1[0]))
    lon1 = radians(float(loc1[1]))
    
    lat2 = radians(float(loc2[0]))
    lon2 = radians(float(loc2[1]))
    
    #calculate the distance
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2( sqrt(a), sqrt(1-a) )
    d = R * c # where R is the radius of the Earth
    return d

## 1. Load road and bridge data (N1)

In [4]:
df = pd.read_csv("_roads3.csv")
df_r = df[df["road"] == "N1"][:]
df = pd.read_excel("BMMS_overview.xlsx")
df_b = df[df["road"] == "N1"][:]
dup = (df_b.sort_values("km").duplicated("km", keep = "first")
       |
       df_b.sort_values("km").duplicated("km", keep = "last"))
df_b.loc[dup].sort_values("km")

new_df = pd.DataFrame()
for i in range(len(df_b.loc[dup]["km"].unique())):
    df_ele = df_b.loc[dup][df_b.loc[dup]["km"] == df_b.loc[dup]["km"].unique()[i]]
    df_ele.fillna(method = "ffill", inplace = True)
    df_ele.fillna(method = "bfill", inplace = True)

    new_df = pd.concat([new_df, df_ele])
# drop duplicates and keep the lower entry, as they are assumed to be recored more recent.
df_b = df_b.drop_duplicates("km", keep = "last").sort_values("km")

# Select Dhaka ~ Chittagong
df_r = df_r.iloc[:565]
df_b = df_b.iloc[:162]

## 2. Convert road/bridge data into the form readable in Simio

In [5]:
def create_data(df_r, df_b, nrow = False):
    """
    Extract necessary information from road/bridge data
    1. Type (Bridge/Road), LRP
    2. Longitude, Latitude, Chainage, Distance
    3. Bridge condition(A/B/C/D) and length
    """
    road = pd.concat([df_r[["lon", "lat", "chainage", "lrp"]],
                      pd.DataFrame(("Road "*len(df_r)).split(), columns = ["type"])], axis = 1)

    bridge = df_b[["lon", "lat", "chainage", "LRPName", "length", "condition", "length"]]
    bridge.columns = ['lon', 'lat', 'chainage', 'lrp', 'BriLen', 'BriCond', 'type']
    bridge["type"] = "Bridge"
    
    transfernode = pd.DataFrame(data = bridge.values, index = bridge.index, columns = bridge.columns)
    transfernode["type"] = "Output@Bridge"
    transfernode["BriLen"] = None

    df = pd.concat([road, bridge, transfernode]).sort_values(by = ["chainage", "type"])
    df.reset_index(inplace = True)
    df["Object Name"] = df["lrp"].values
    df["Object Name"] = df[["type", "Object Name"]].apply(lambda x: "".join(["Output@Bridge", x[1]])
                                                          if x[0][0] == "O"
                                                          else x[1],axis = 1).values
    
    df["Object Name"] = df[["type", "Object Name"]].apply(lambda x: "".join(["Bridge", x[1]])
                                                          if x[0] == "Bridge"
                                                          else x[1],axis = 1).values
    
    df["distance"] = [1e3 * distance(df[["lat", "lon"]].iloc[i],
                                     df[["lat", "lon"]].iloc[i+1]) for i in range(len(df) - 1)] + [None]
    if type(nrow) == int:
        df = df.loc[:nrow]
    elif not nrow:
        df = df
    return df
data = create_data(df_r, df_b, 21)

C:\Users\markhupkens\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


#### Distance Check

In [6]:
create_data(df_r, df_b)["distance"].sum()

C:\Users\markhupkens\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


231634.91190303524

### 35 Segments → 4 Segments (Highly Aggregated)

In [7]:
def agg_segments(df_traffic, idx = [0,6,23,25,36]):
    # segment starts/ends
    seg_start = idx[:-1]
    seg_end = idx[1:]
    func = np.vectorize(lambda x: x in seg_start)
    seg_start = func(df_traffic.index.values)
    func = np.vectorize(lambda x: x in seg_end)
    seg_end = func(df_traffic.index.values)
    seg_end[-1] = True
    df_traffic["seg_start"] = seg_start
    df_traffic["seg_end"] = seg_end
    
    # assign segments
    df_traffic["segment"] = np.repeat(range(1,5), np.diff(np.array(idx)))
    seg_ori = np.array(df_traffic["Link No"].values)
    seg_new = "N1_" + df_traffic["segment"].values.astype(str).astype(object)
    df_traffic["Link No"] = seg_new
    df_traffic["segment"] = seg_ori
    return df_traffic

df_traffic.head()

NameError: name 'df_traffic' is not defined

### Add Road Segments (Traffic Data)

In [52]:
def add_segments(df_r, df_b, traffic_file = "N1_traffic.csv", n = 35):
#     n=35: Dhaka~Chittagong
    df_traffic = pd.read_csv(traffic_file, index_col = 0).loc[:n]
#     Aggregate into 4 Segments
    df_traffic = agg_segments(df_traffic)
#     Load Road, Bridge Data
    data = create_data(df_r, df_b)
#     Start building!
    df = pd.DataFrame(data = data.values, columns = data.columns)    
#     Boolean (whether segment starts/ends)
    seg_start = df_traffic[df_traffic["seg_start"]]["Chainage_Start"].values
    seg_end = df_traffic[df_traffic["seg_end"]]["Chainage_Start"].values
    seg_end = np.append(seg_end[:-1], df["chainage"].tail(1).values)
    seg = np.append(seg_start, seg_end[-1])
    
    df["seg_start"] = df["chainage"].apply(lambda x: x in seg_start).values
    df["seg_end"] = df["chainage"].apply(lambda x: x in seg_end).values

#     if a segment starts (or ends) -> node
#     if not -> vertice
    df["vertice"] = df["chainage"].apply(lambda x: "transfernode" if x in seg else "vertice").values
    
#     Bridge
    idx_bridge = df.index[df["type"].apply(lambda x: x == "Bridge")]
#     Bridge is bridge
    df.loc[idx_bridge, "vertice"] = "bridge"
#     neighboring point to bridge -> node
    df.loc[idx_bridge - 1, "vertice"] = "basicnode"
    df.loc[idx_bridge + 1, "vertice"] = "basicnode"
    
    df.loc[df.index[df["seg_start"]],"vertice"] = "transfernode"
    df.loc[df.index[df["seg_end"]],"vertice"] = "transfernode"
    
#     Paths which have to be made in Simio
    paths = []
    i = 0
    for val in df["vertice"].values:
        if val in ["transfernode", "basicnode"] :
            i = i + 1
        paths.append(str(i))
#     Give a name to each path (Path0, Path1, Path2,...)
    df["path"] = np.repeat("Path", len(paths)).astype(np.object) + np.array(paths, dtype=np.object)
    
#     distance value of vertice -> aggregate into node
    d = {}
    for path in df["path"].unique():
        idx = df.index[df["path"] == path]
        df_dis = df.loc[idx, "distance"]
        dis = df_dis.sum()
        d[path] = dis
    df["distance"] = df["path"].map(d)
    df = df.reset_index(drop = True)
#     print(df_traffic[["Link No", "segment"]])
#     Set segments
    n_seg = df_traffic["Link No"].nunique()
    where_start_end = df["seg_start"] | df["seg_end"]
    where_start_end = df[where_start_end].index
    
    n_repeat = np.diff(where_start_end)
    n_repeat[-1] = n_repeat[-1] + 1 #compensate the last element
    df["segment"] = np.repeat(range(1,5),n_repeat)
    # string join ("1" -> "N1_1")
    df["segment"] = df["segment"].apply(lambda x: "".join(["N1_", str(x)]))
    df_tra = df_traffic.iloc[:, [0]+list(range(9,25))].groupby("Link No").mean().reset_index()
    return pd.merge(df, df_tra, left_on = "segment", right_on="Link No"), df_traffic[["Link No", "segment"]]

### Link Sheet

In [33]:
# Define Function
def create_link(data):
    # data frame for creating object and link excel sheets
    df_obj = data[data["vertice"] != "vertice"]

    # Blank data frame
    col = "Link Class;Link Name;From Node;To Node;Network;Type;DrawnToScale;LogicalLength".split(";")
    df_link = pd.DataFrame(columns=col)

    # From node
    df_link["From Node"] = df_obj[df_obj["type"] != "Bridge"]["Object Name"].values[:-1]
    df_link["To Node"] = df_obj[df_obj["type"] != "Bridge"]["Object Name"].values[1:]
    # Change string "Bridge" -> "Input@Bridge"
    df_link["To Node"] = df_link["To Node"].apply(lambda x: "".join(["Input",x[6:]]) if x[0] == "O" else x).values

    # assign values 1
    df_link["Link Name"] = df_obj[df_obj["type"] != "Bridge"]["path"].values[:-1]
    df_link["LogicalLength"] = df_obj[df_obj["type"] != "Bridge"]["distance"].values[:-1]
    df_link["Network"] = df_obj[df_obj["type"] != "Bridge"]["segment"].values[:-1]

    # assign values 2
    df_link["Link Class"] = "Path"
    df_link["DrawnToScale"] = "False"
    df_link["Type"] = "Unidirectional"
    return df_link

#### Distance Check (Fine)

In [34]:
create_link(data)["LogicalLength"].sum()

231634.9119030353

### Object Sheet

In [35]:
def create_obj(data, multi = 100, ng = -1):
    # drop vertice
    df_obj = data[data["vertice"] != "vertice"]
    # drop output@bridge
    df_obj = df_obj[df_obj["type"] != "Output@Bridge"]
    
    col = ['Object Class', 'Object Name', 'X', 'Y', 'Z', 'Length', 'Width', 'Height', 'BriLen', 'BriCond']
    df_objs = pd.DataFrame(columns = col)
    
    df_objs["Object Class"] = df_obj["vertice"].apply(lambda x: "TransferNode" if x == "transfernode"
                                                  else "BasicNode" if x == "basicnode"
                                                  else "Bridge").values
    df_objs["Object Name"] = df_obj["Object Name"].values

    df_objs["X"] = df_obj["lon"].apply(lambda x: x * multi).values
    df_objs["Y"] = 0
    df_objs["Z"] = df_obj["lat"].apply(lambda x: x * multi * ng).values
    df_objs["BriLen"] = df_obj["BriLen"].values
    df_objs["BriCond"] = df_obj["BriCond"].values
    df_objs["EnteredAddOnProcess"] = df_objs["Object Class"].apply(lambda x: "Process_StartPointSegment"
                                                               if x == "TransferNode"
                                                               else np.nan)
    return df_objs

### Vertice Sheet

In [36]:
def create_vertice(data, multi = 100, ng = -1):
    df_vertice = data[data["vertice"] == "vertice"]
    col = ["Link Name","Vertex X", "Vertex Y","Vertex Z"]
    df_ver = pd.DataFrame(columns=col)
    df_ver["Link Name"] = df_vertice["path"].values

    df_ver["Vertex X"] = df_vertice["lon"].apply(lambda x: x * multi).values
    df_ver["Vertex Y"] = 0
    df_ver["Vertex Z"] = df_vertice["lat"].apply(lambda x: x * multi * ng).values
    return df_ver

## Export Data

In [69]:
data, data_traffic = add_segments(df_r, df_b, traffic_file="N1_traffic.csv")
sheet_obj = create_obj(data, multi = 10000)
sheet_link = create_link(data)
sheet_vertice = create_vertice(data, multi = 10000)
# save to excel file
writer = pd.ExcelWriter('SimioObjects.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
sheet_obj.to_excel(writer, sheet_name='Objects1', index = False)
sheet_link.to_excel(writer, sheet_name='Links1', index = False)
sheet_vertice.to_excel(writer, sheet_name='Vertices1', index = False)

writer.save()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


### 4 Segments and 35 segments

In [70]:
data_traffic["segment"] = data_traffic["segment"].apply(lambda x: "".join([x,";"])).values
data_traffic = data_traffic.groupby("Link No").sum()

In [76]:
data_traffic["segment"].apply(lambda x: x.split(";")).to_csv("agg_segment.csv")

### Total Distance of each Segment

In [38]:
sheet_link.groupby("Network").sum()

,LogicalLength
Network,
N1_1,15757.742678
N1_2,89256.084574
N1_3,32466.820604
N1_4,94154.264048


In [39]:
# Speed of Each Entity
spd_truck = 5
spd_bus = 6
spd_other = 2

In [40]:
expected_time = sheet_link.groupby("Network").sum().reset_index()
expected_time["truck"] = expected_time["LogicalLength"] / spd_truck / 3600
expected_time["bus"] = expected_time["LogicalLength"] / spd_bus / 3600
expected_time["other"] = expected_time["LogicalLength"] / spd_other / 3600
expected_time

,Network,LogicalLength,truck,bus,other
0,N1_1,15757.742678,0.875430,0.729525,2.188575
1,N1_2,89256.084574,4.958671,4.132226,12.396678
2,N1_3,32466.820604,1.803712,1.503094,4.509281
3,N1_4,94154.264048,5.230792,4.358994,13.076981


In [41]:
expected_time.to_csv("expected_time.csv", index = False)

### Data Tables (Segments, Tally, States)

In [157]:
def data_table(data, string):
    table = data.loc[data[data["seg_end"]].index - 1]["path"].reset_index()
    table["lrp"] = data[data["seg_start"]]["lrp"].values
    table["TimeInSys"] = np.arange(1, 1+len(table)).astype(str)
    table["TimeInSys"] = table["TimeInSys"].apply(lambda x: "".join(["TalSeg",x,"_"]))
    table["NumInSys"] = np.arange(1, 1+len(table)).astype(str)
    table["NumInSys"] = table["NumInSys"].apply(lambda x: "".join(["NumSeg",x,"_"]))
    table = table.drop("index", axis = 1)
    table.columns = ["LastPath", "Start Node", "TimeInSys", "NumInSys"]

    table["TimeInSys"] = table["TimeInSys"].apply(lambda x: "".join([x, string]))
    table["NumInSys"] = table["NumInSys"].apply(lambda x: "".join([x, string]))
    return table

In [159]:
data_table(data, "Bus").to_csv("table_bus.csv", index = False)
data_table(data, "Truck").to_csv("table_truck.csv", index = False)
data_table(data, "Other").to_csv("table_other.csv", index = False)

### Transfer Table
    for creating transportations and distributing to every segment

In [17]:
transfer_table = pd.DataFrame(data.iloc[:, [8]][data["seg_start"]].values,
                              columns = ["Destination"])
start = np.where(data.columns == "Heavy Truck")[0][0]
transfer_table = transfer_table.merge(data.iloc[:, start:start+12][data["seg_start"]].reset_index(drop = True), left_index = True, right_index = True)
transfer_table = pd.DataFrame(transfer_table.set_index("Destination").stack()).reset_index()
transfer_table.columns = ["Destination", "EntityType", "Traffic"]
transfer_table["EntityType"].unique()
transfer_table["category"] = transfer_table["EntityType"].apply(lambda x: "Truck" if "Truck" in x
                                                                else "Bus" if "Bus" in x
                                                                else "other" if "Rickshaw" in x
                                                                else "other" if "Cycle" in x
                                                                else np.nan)
# Drop out-of-scope (Util, Car, Cart)
transfer_table.dropna(inplace = True)
transfer_table = transfer_table.groupby(["category", "Destination"]).sum().reset_index()
# Normalize (Scale Down) by minimum value
print("NORMALIZER is", transfer_table["Traffic"].min())
transfer_table["Traffic"] = transfer_table["Traffic"].apply(lambda x: np.round(x/transfer_table["Traffic"].min()))
transfer_table

NORMALIZER is 3344.5


,category,Destination,Traffic
0,Bus,LRP016a,2.0
1,Bus,LRP108c,1.0
2,Bus,LRP142a,2.0
3,Bus,LRPS,4.0
4,Truck,LRP016a,4.0
5,Truck,LRP108c,3.0
6,Truck,LRP142a,4.0
7,Truck,LRPS,4.0
8,other,LRP016a,1.0
9,other,LRP108c,1.0


In [123]:
transfer_table.to_csv("transfer_table.csv", index = False)